In [1]:
from mlrun import get_or_create_ctx, mlconf
from mlrun.artifacts import TableArtifact

In [2]:
# nuclio: ignore
mlrunctx = get_or_create_ctx('mlrun')

[mlrun] 2020-03-14 20:28:53,206 logging run results to: http://10.196.67.76:8080


In [3]:
# Import needed functions/classes
from netapp_ontap import config as netappConfig
from netapp_ontap.host_connection import HostConnection as NetAppHostConnection
from netapp_ontap.resources import Volume, Snapshot
from netapp_ontap import config, utils

from datetime import datetime
import json
import sys, subprocess
import os


In [7]:
def netappSnapshot(
    context,
    ontapClusterMgmtHostname,
    ontapClusterAdminUsername,
    ontapClusterAdminPassword,
    sourceVolumeName,
    verifySSLCert
):
    is_simulated = context.get_param('NETAPP_SIM',False)
    NETAPP_MOUNT_PATH = context.get_param('NETAPP_MOUNT_PATH','/netapp')
    print("SIMULATED",is_simulated,ontapClusterMgmtHostname,context)
    if is_simulated:
        snap_path = os.path.join(NETAPP_MOUNT_PATH)
        context.log_result('snapVolumeDetails',snap_path)
        context.logger.info('snap volume name: %s'%snap_path)
        return snap_path
    print("====>",ontapClusterMgmtHostname)  
    # Configure connection to ONTAP cluster/instance
    netappConfig.CONNECTION = NetAppHostConnection(
        host = ontapClusterMgmtHostname,
        username = ontapClusterAdminUsername,
        password = ontapClusterAdminPassword,
        verify = verifySSLCert
    )
   
    # Convert pv name to ONTAP volume name
    # The following will not work if you specified a custom storagePrefix when creating your
    #   Trident backend. If you specified a custom storagePrefix, you will need to update this
    #   code to match your prefix.
    #volumeName = 'trident_%s' % pvName.replace("-", "_")
    volumeName = sourceVolumeName
    
    
    context.logger.info('Source ONTAP volume name: %s'%sourceVolumeName)
    
    # Create snapshot; print API response
    volume = Volume.find(name = volumeName)
    timestamp = datetime.today().strftime("%Y%m%d_%H%M%S")
    snapshot = Snapshot.from_dict({
        'name': 'kfp_%s' % timestamp,
        'comment': 'Snapshot created by a Kubeflow pipeline',
        'volume': volume.to_dict()
    })
    response = snapshot.post()
    print("\nAPI Response:")
    print(response.http_response.text)
 
    # Retrieve snapshot details
    snapshot.get()
 
    # Convert snapshot details to JSON string and print
    snapshotDetails = snapshot.to_dict()
    print("\nSnapshot Details:")
    print(json.dumps(snapshotDetails, indent=2))
 
    snap_path = os.path.join(NETAPP_MOUNT_PATH,'.snapshot',snapshotDetails['name'])
    context.log_result('snapVolumeDetails',snap_path)
    # Return name of newly created snapshot
    return snap_path


In [ ]:
def get_latest_dataset(context,snap_path):
    metrics_table=os.path.join(snap_path,context.get_param('metrics_table', 'metrics_table'))
    # Get parquet files
    mpath = [os.path.join(metrics_table, file) for file in os.listdir(metrics_table)]
    
    # Get latest filename
    latest = max(mpath, key=os.path.getmtime)
    context.log_result('training_parquet_file',latest)
    context.logger.debug(f'Reading data from: {latest}')
    
    return ""

In [13]:
def handler(context,
    ontapClusterMgmtHostname  = 'localhost',
    ontapClusterAdminUsername = 'admin',
    ontapClusterAdminPassword = 'NetApp!23',
    sourceVolumeName = 'trident_default_tenant_basic_08204',
    verifySSLCert: bool = False
):
    # snap NetApp Volume
    snap_path  = netappSnapshot(context,ontapClusterMgmtHostname,ontapClusterAdminUsername,ontapClusterAdminPassword,sourceVolumeName,verifySSLCert)
    
    get_latest_dataset(context,snap_path)
    # Get latest data set for training